# Locker Open Detector

## A. Enivironment Setup

In [1]:
# Install dependencies
%pip install ultralytics opencv-python pillow scikit-learn pyyaml

Note: you may need to restart the kernel to use updated packages.


In [2]:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [3]:
# Import libraries with better organization

# Gdrive
# from google.colab import drive

# System libraries
from os import path, listdir, makedirs
from shutil import copyfile
from time import sleep
from base64 import b64encode
from pathlib import Path
from typing import List, Tuple, Dict, Union

# Data processing
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import json
import yaml

# Image & video processing
import cv2
from PIL import Image
from matplotlib import pyplot as plt

# Deep learning
from ultralytics import YOLO
from torch import __version__, cuda, version, backends

# Display
from IPython.display import HTML, display
from glob import glob

In [4]:
# Cek versi PyTorch
print(f"PyTorch Version: {__version__}")

# Cek ketersediaan CUDA
print(f"CUDA Available: {cuda.is_available()}")

# Cek versi CUDA yang digunakan PyTorch
if cuda.is_available():
    print(f"CUDA Version: {version.cuda}")
    
    # Cek perangkat CUDA
    current_device = cuda.current_device()
    print(f"Current CUDA Device: {cuda.get_device_name(current_device)}")
    print(f"Device Capability: {cuda.get_device_capability()}")
    print(f"Device Count: {cuda.device_count()}")
    
    # Cek memori GPU
    print(f"Total Memory: {cuda.get_device_properties(current_device).total_memory / 1e9:.2f} GB")
    print(f"Allocated Memory: {cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"Reserved Memory: {cuda.memory_reserved() / 1e9:.2f} GB")

# Cek versi cuDNN (jika tersedia)
try:
    print(f"cuDNN Version: {backends.cudnn.version()}")
    print(f"cuDNN Enabled: {backends.cudnn.enabled}")
except:
    print("cuDNN version tidak dapat dideteksi")

PyTorch Version: 2.7.1+cu118
CUDA Available: True
CUDA Version: 11.8
Current CUDA Device: NVIDIA GeForce MX350
Device Capability: (6, 1)
Device Count: 1
Total Memory: 2.15 GB
Allocated Memory: 0.00 GB
Reserved Memory: 0.00 GB
cuDNN Version: 90100
cuDNN Enabled: True


## B. Data Preparation

In [5]:
# Connect google drive
# drive.mount('/content/drive')

In [6]:
# Base directories
project_dir: Path = Path(".")
dataset_dir: Path = project_dir / "dataset"
images_dir: Path = dataset_dir / "images"
labels_dir: Path = dataset_dir / "labels"
models_dir: Path = project_dir / "models"

# Training directories
train_img_dir: Path = images_dir / "train"
val_img_dir: Path = images_dir / "val"
train_lbl_dir: Path = labels_dir / "train"
val_lbl_dir: Path = labels_dir / "val"

# Create directories if they don't exist
for directory in [images_dir, labels_dir, train_img_dir, val_img_dir, train_lbl_dir, val_lbl_dir]:
    directory.mkdir(exist_ok=True, parents=True)

# List dataset files
print(f"Files in '{dataset_dir}':")
for item in dataset_dir.iterdir():
    print(f"- {item.name}")

Files in 'dataset':
- dataset.csv
- dataset.yaml
- images
- labels
- locker.mp4
- models


### Video Processing

In [7]:
# Open and display video with error handling
video_path: Path = dataset_dir / "locker.mp4"
print(f"Video path: {video_path}")

try:
    with open(video_path, "rb") as video_file:
        data: bytes = video_file.read()
        video_base64: str = b64encode(data).decode()

        video_html: str = f"""
          <video width="480" height="360" controls>
            <source src="data:video/mp4;base64,{video_base64}" type="video/mp4">
            Your browser does not support the video tag.
          </video>
        """
        display(HTML(video_html))
except FileNotFoundError:
    print(f"❌ Video file not found: {video_path}")

Video path: dataset\locker.mp4


In [8]:
# Extract frames with progress tracking
def extract_frames(video_path: Path, output_dir: Path, sample_rate: int = 10) -> int:
    """
    Extract frames from video at specified sample rate
    
    Args:
        video_path: Path to video file
        output_dir: Directory to save extracted frames
        sample_rate: Extract 1 frame for every 'sample_rate' frames
        
    Returns:
        Number of frames extracted
    """
    # Ensure output directory exists
    Path(output_dir).mkdir(exist_ok=True, parents=True)
    
    # Open video
    cap: cv2.VideoCapture = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise ValueError(f"Failed to open video: {video_path}")
    
    # Get video properties
    frame_count: int = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps: int = int(cap.get(cv2.CAP_PROP_FPS))

    print(f"Video info: {frame_count} frames, {fps} FPS")
    print(f"Extracting approximately {frame_count/sample_rate:.0f} frames...")
    
    # Calculate sampling interval
    frame_interval: int = max(1, int(fps / sample_rate))
    
    # Extract frames
    frame_number: int = 0
    saved_count: int = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        # Save frame at specified intervals
        if frame_number % frame_interval == 0:
            frame_path: Path = output_dir / f"frame_{frame_number:04d}.png"
            cv2.imwrite(str(frame_path), frame)
            saved_count += 1
            
            # Print progress every 10 frames
            if saved_count % 10 == 0:
                print(f"Progress: {saved_count} frames extracted", end="\r")
                
        frame_number += 1
    
    # Release resources
    cap.release()
    print(f"\nExtracted {saved_count} frames to {output_dir}")
    return saved_count

# Extract frames
extract_frames(video_path, images_dir)

Video info: 949 frames, 59 FPS
Extracting approximately 95 frames...
Progress: 190 frames extracted
Extracted 190 frames to dataset\images


190

## C. Dataset Loading

In [9]:
# Load dataset with validation
def load_dataset(csv_path: Path, image_dir: Path) -> pd.DataFrame:
    """Load and validate dataset from CSV"""
    try:
        # Read CSV
        df: pd.DataFrame = pd.read_csv(csv_path)
        print(f"Loaded {len(df)} records from {csv_path}")
        
        # Fix image paths - extract only the frame_XXXX.png part
        df['original_path'] = df['image'].copy()
        
        # Extract just the frame_XXXX.png part from the filename
        df['image'] = df['image'].apply(
            lambda x: path.join(image_dir, 
                               x.split('/')[-1].split('-')[-1] if '-' in x 
                               else path.basename(x))
        )
        
        # Verify images exist
        missing: pd.DataFrame = df[~df['image'].apply(path.exists)]
        if len(missing) > 0:
            print(f"⚠️ Warning: {len(missing)} images not found")
            print(missing['image'].head())
        else:
            print("✅ All image files found successfully!")
        
        return df
    except Exception as e:
        print(f"❌ Error loading dataset: {str(e)}")
        return pd.DataFrame()

# Load dataset
dataset_csv: Path = dataset_dir / "dataset.csv"
df: pd.DataFrame = load_dataset(dataset_csv, images_dir)

if not df.empty:
    display(df.head())

Loaded 97 records from dataset\dataset.csv
✅ All image files found successfully!


,annotation_id,annotator,created_at,id,image,label,lead_time,updated_at,original_path
0,225,1,2025-09-16T10:27:50.656228Z,39,dataset\images\frame_0190.png,"[{""x"":0,""y"":5.551151746803923,""width"":6.593406...",141.537,2025-09-16T10:27:50.656228Z,/data/upload/5/9f34d718-frame_0190.png
1,226,1,2025-09-16T10:28:43.853569Z,40,dataset\images\frame_0195.png,"[{""x"":6.318681318681318,""y"":4.90107718368588,""...",46.396,2025-09-16T10:28:43.853569Z,/data/upload/5/2f661a75-frame_0195.png
2,227,1,2025-09-16T10:29:47.293860Z,41,dataset\images\frame_0200.png,"[{""x"":6.684981684981685,""y"":5.0635958244653905...",54.508,2025-09-16T10:29:47.293860Z,/data/upload/5/10f97039-frame_0200.png
3,228,1,2025-09-16T10:30:47.993295Z,42,dataset\images\frame_0205.png,"[{""x"":6.501831501831503,""y"":4.576039902126859,...",57.009,2025-09-16T10:30:47.993295Z,/data/upload/5/b0c4eb42-frame_0205.png
4,229,1,2025-09-16T10:50:54.672433Z,43,dataset\images\frame_0210.png,"[{""x"":6.77655677655678,""y"":5.388633106024414,""...",1201.955,2025-09-16T10:50:54.672433Z,/data/upload/5/90ae20a7-frame_0210.png


## D. Data Splitting

In [10]:
def split_dataset(
        df: pd.DataFrame, 
        train_img_dir: Path, 
        val_img_dir: Path, 
        train_lbl_dir: Path, 
        val_lbl_dir: Path, 
        test_size: float = 0.2
    ) -> Tuple[List[str], List[str]]:
    """Split dataset into training and validation sets"""
    # Get image paths
    image_paths: List[str] = df['image'].tolist()
    
    # Split into train/val
    train_images, val_images = train_test_split(
        image_paths, 
        test_size=test_size, 
        random_state=42
    )
    
    # Copy images and labels to respective directories
    copy_count: Dict = {'train': 0, 'val': 0}
    
    for img_path in train_images:
        img_dest = path.join(train_img_dir, path.basename(img_path))
        copyfile(img_path, img_dest)
        copy_count['train'] += 1
        
    for img_path in val_images:
        img_dest = path.join(val_img_dir, path.basename(img_path))
        copyfile(img_path, img_dest)
        copy_count['val'] += 1
    
    print(f"✅ Copied {copy_count['train']} training images and {copy_count['val']} validation images")
    print(f"✅ Prepared label directories: {train_lbl_dir} and {val_lbl_dir}")
    
    return train_images, val_images

# Split dataset
train_images, val_images = split_dataset(df, train_img_dir, val_img_dir, train_lbl_dir, val_lbl_dir)

✅ Copied 77 training images and 20 validation images
✅ Prepared label directories: dataset\labels\train and dataset\labels\val


## E. YOLO Format Conversion

In [11]:
def convert_to_yolo_format(
        df: pd.DataFrame, 
        train_images: List[str], 
        val_images: List[str], 
        train_lbl_dir: Path, 
        val_lbl_dir: Path
    ) -> None:
    """Convert annotations to YOLO format"""
    stats: Dict = {'success': 0, 'errors': 0, 'missing': 0}
    
    for index, row in df.iterrows():
        image_path = row['image']
        
        # Parse JSON labels
        try:
            label_data: List[Dict] = json.loads(row['label'])
        except json.JSONDecodeError:
            stats['errors'] += 1
            print(f"❌ Error parsing JSON in line {index}")
            continue
            
        # Get image dimensions
        try:
            with Image.open(image_path) as img:
                image_width, image_height = img.size
        except FileNotFoundError:
            stats['missing'] += 1
            print(f"❌ Image not found: {image_path}")
            continue
            
        # Determine output path based on train/val split
        basename: Path = path.basename(image_path).replace('.png', '.txt')
        if image_path in train_images:
            txt_path = path.join(train_lbl_dir, basename)
        elif image_path in val_images:
            txt_path = path.join(val_lbl_dir, basename)
        else:
            continue
            
        # Write YOLO format labels
        with open(txt_path, 'w') as f:
            for bbox in label_data:
                try:
                    # Extract coordinates
                    x: float = bbox['x']
                    y: float = bbox['y']
                    width: float = bbox['width']
                    height: float = bbox['height']
                    
                    # Set class ID (0 for open, 1 for closed)
                    class_id: int = bbox.get('class_id', 0)
                    
                    # Convert to YOLO format (normalized)
                    center_x: float = (x + width / 2) / image_width
                    center_y: float = (y + height / 2) / image_height
                    norm_width: float = width / image_width
                    norm_height: float = height / image_height
                    
                    # Write to file
                    f.write(f"{class_id} {center_x:.6f} {center_y:.6f} {norm_width:.6f} {norm_height:.6f}\n")
                except KeyError as e:
                    print(f"⚠️ Missing field in bounding box: {e}")
        
        stats['success'] += 1
    
    print(f"✅ Converted {stats['success']} annotations to YOLO format")
    print(f"❌ Errors: {stats['errors']} JSON parse errors, {stats['missing']} missing images")

# Convert annotations
convert_to_yolo_format(df, train_images, val_images, train_lbl_dir, val_lbl_dir)

✅ Converted 97 annotations to YOLO format
❌ Errors: 0 JSON parse errors, 0 missing images


## F. Configuration & Training

In [12]:
def create_dataset_yaml(train_dir: Path, val_dir: Path, output_path: Path) -> Dict:
    """Create YAML configuration file for YOLOv8"""
    abs_dataset_dir = dataset_dir.absolute()

    yaml_content: Dict[str, Union[str, int, List[str]]] = {
        'path': str(abs_dataset_dir),
        'train': 'images/train',
        'val': 'images/val',
        'nc': 2,
        'names': ['open', 'close']
    }
    
    with open(output_path, 'w') as f:
        yaml.dump(yaml_content, f, sort_keys=False)
        
    print(f"✅ Dataset configuration created at {output_path}")
    return yaml_content

# Create dataset YAML
yaml_path: Path = dataset_dir / "dataset.yaml"
config: Dict[str, Union[str, int, List[str]]] = create_dataset_yaml(train_img_dir, val_img_dir, yaml_path)

# Display configuration
print("\nYAML Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

✅ Dataset configuration created at dataset\dataset.yaml

YAML Configuration:
  path: d:\Salman\Kerja\Pantona\Locker\dataset
  train: images/train
  val: images/val
  nc: 2
  names: ['open', 'close']


## G. Training & Evaluation

In [ ]:
def train_model(yaml_path: Path, model_path: Path, epochs: int, img_size: int):
    """Train YOLOv11 model"""
    try:
        # Set device (GPU or CPU)
        device = 0 if cuda.is_available() else 'cpu'  # Use device=0 for the first GPU
        print(f"Using device: {'CUDA' if device == 0 else 'CPU'}")

        # Load model (use pretrained or download if not available)
        if model_path and path.exists(model_path):
            model: YOLO = YOLO(model_path)
            print(f"Loaded model from: {model_path}")
        else:
            # Use YOLOv11m if custom model not available
            model: YOLO = YOLO('yolo11m.pt')
            print("Using default YOLO11m model")
        
        # Train model
        print(f"Starting training for {epochs} epochs...")
        results = model.train(
            data=yaml_path,
            epochs=epochs,
            imgsz=img_size,
            verbose=True,
            plots=True,
            device=device
        )
        
        print(f"✅ Training complete. Results saved to {results}")
        return results
    except Exception as e:
        print(f"❌ Training error: {str(e)}")
        return None

# Train model 
results = train_model(
    yaml_path=yaml_path,
    model_path=models_dir / "models/yolo11m.pt",
    epochs=5,
    img_size=128,
)

Using device: CUDA
Using default YOLO11m model
Starting training for 5 epochs...
New https://pypi.org/project/ultralytics/8.3.201 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.174  Python-3.11.9 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce MX350, 2048MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset\dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=128, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0

train: Scanning D:\Salman\Kerja\Pantona\Locker\dataset\labels\train.cache... 77 images, 0 backgrounds, 0 corrupt: 100%|██████████| 77/77 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 25.81.9 MB/s, size: 483.8 KB)


val: Scanning D:\Salman\Kerja\Pantona\Locker\dataset\labels\val.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]


Plotting labels to runs\detect\train14\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.0005), 112 bias(decay=0.0)
Image sizes 128 train, 128 val
Using 8 dataloader workers
Logging results to runs\detect\train14
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

        5/5     0.977G    0.01242      2.163   0.003029         19        128: 100%|██████████| 5/5 [00:01<00:00,  3.00it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  4.84it/s]

                   all         20         55          0          0          0          0



5 epochs completed in 0.008 hours.
Optimizer stripped from runs\detect\train14\weights\last.pt, 40.5MB
Optimizer stripped from runs\detect\train14\weights\best.pt, 40.5MB

Validating runs\detect\train14\weights\best.pt...
Ultralytics 8.3.174  Python-3.11.9 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce MX350, 2048MiB)
YOLO11m summary (fused): 125 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  4.77it/s]


                   all         20         55          0          0          0          0
                  open         20         55          0          0          0          0
Speed: 0.1ms preprocess, 3.5ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs\detect\train14
✅ Training complete. Results saved to ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000024C5F23B310>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,


### Visualization

In [14]:
def visualize_training_results(results_dir: Path) -> None:
    """Visualization training result YOLO"""
    
    # Plot metrics
    metrics_path: Path = results_dir / 'results.csv'
    if metrics_path.exists():
        # Plot metrics
        metrics_df: pd.DataFrame = pd.read_csv(metrics_path)
        
        # Debug: Print available columns
        print("Available columns in the results CSV:")
        print(metrics_df.columns.tolist())
        
        # Identify relevant columns
        epoch_col: str = 'epoch'
        box_loss_train_col: str = 'train/box_loss'
        box_loss_val_col: str = 'val/box_loss'
        cls_loss_train_col: str = 'train/cls_loss'
        cls_loss_val_col: str = 'val/cls_loss'
        dfl_loss_train_col: str = 'train/dfl_loss'
        dfl_loss_val_col:str = 'val/dfl_loss'
        map50_col: str = 'metrics/mAP50(B)'
        map50_95_col: str = 'metrics/mAP50-95(B)'

        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Plot Box Loss
        if all(col in metrics_df.columns for col in [epoch_col, box_loss_train_col, box_loss_val_col]):
            axes[0, 0].plot(metrics_df[epoch_col], metrics_df[box_loss_train_col], label='train_box_loss', color='blue')
            axes[0, 0].plot(metrics_df[epoch_col], metrics_df[box_loss_val_col], label='val_box_loss', color='orange', linestyle='--')
            axes[0, 0].set_title('Box Loss')
            axes[0, 0].set_xlabel('Epoch')
            axes[0, 0].set_ylabel('Loss')
            axes[0, 0].legend()
            axes[0, 0].grid(True)
        else:
            axes[0, 0].text(0.5, 0.5, 'Box Loss - Data Not Available', ha='center', va='center')
            axes[0, 0].set_title('Box Loss - Data Not Available')
            
        # Plot Classification Loss
        if all(col in metrics_df.columns for col in [epoch_col, cls_loss_train_col, cls_loss_val_col]):
            axes[0, 1].plot(metrics_df[epoch_col], metrics_df[cls_loss_train_col], label='train_cls_loss', color='green')
            axes[0, 1].plot(metrics_df[epoch_col], metrics_df[cls_loss_val_col], label='val_cls_loss', color='red', linestyle='--')
            axes[0, 1].set_title('Classification Loss')
            axes[0, 1].set_xlabel('Epoch')
            axes[0, 1].set_ylabel('Loss')
            axes[0, 1].legend()
            axes[0, 1].grid(True)
        else:
            axes[0, 1].text(0.5, 0.5, 'Classification Loss - Data Not Available', ha='center', va='center')
            axes[0, 1].set_title('Classification Loss - Data Not Available')
        
        # Plot DFL Loss
        if all(col in metrics_df.columns for col in [epoch_col, dfl_loss_train_col, dfl_loss_val_col]):
            axes[1, 0].plot(metrics_df[epoch_col], metrics_df[dfl_loss_train_col], label='train_dfl_loss', color='purple')
            axes[1, 0].plot(metrics_df[epoch_col], metrics_df[dfl_loss_val_col], label='val_dfl_loss', color='brown', linestyle='--')
            axes[1, 0].set_title('DFL Loss')
            axes[1, 0].set_xlabel('Epoch')
            axes[1, 0].set_ylabel('Loss')
            axes[1, 0].legend()
            axes[1, 0].grid(True)
        else:
            axes[1, 0].text(0.5, 0.5, 'DFL Loss - Data Not Available', ha='center', va='center')
            axes[1, 0].set_title('DFL Loss - Data Not Available')
        
        # Plot Mean Average Precision
        if all(col in metrics_df.columns for col in [epoch_col, map50_col, map50_95_col]):
            axes[1, 1].plot(metrics_df[epoch_col], metrics_df[map50_col], label='mAP50', color='cyan')
            axes[1, 1].plot(metrics_df[epoch_col], metrics_df[map50_95_col], label='mAP50-95', color='magenta')
            axes[1, 1].set_title('Mean Average Precision')
            axes[1, 1].set_xlabel('Epoch')
            axes[1, 1].set_ylabel('mAP')
            axes[1, 1].legend()
            axes[1, 1].grid(True)
        else:
            axes[1, 1].text(0.5, 0.5, 'Mean Average Precision - Data Not Available', ha='center', va='center')
            axes[1, 1].set_title('Mean Average Precision - Data Not Available')
        
        plt.tight_layout()
        plt.show(block=False)  # Non-blocking show
        plt.pause(0.1)  # Pause for rendering

    else:
        print(f"⚠️ Results file not found: {metrics_path}")
    
    # Display example predictions dengan error handling
    results_img: Path = results_dir / 'val_batch0_pred.jpg'
    if results_img.exists():
        try:
            fig, ax = plt.subplots(figsize=(12, 8))
            img = plt.imread(results_img)
            ax.imshow(img)
            ax.axis('off')
            ax.set_title('Validation Predictions', fontsize=16)
            plt.tight_layout()
            plt.show(block=False)
            plt.pause(0.1)
        except Exception as e:
            print(f"❌ Error loading prediction image: {e}")
    else:
        print(f"⚠️ Prediction image not found: {results_img}")
        
    # Display confusion matrix
    conf_matrix: Path = results_dir / 'confusion_matrix.png'
    if conf_matrix.exists():
        try:
            fig, ax = plt.subplots(figsize=(10, 8))
            img = plt.imread(conf_matrix)
            ax.imshow(img)
            ax.axis('off')
            ax.set_title('Confusion Matrix', fontsize=16)
            plt.tight_layout()
            plt.show(block=False)
            plt.pause(0.1)
        except Exception as e:
            print(f"❌ Error loading confusion matrix: {e}")
    else:
        print(f"⚠️ Confusion matrix not found: {conf_matrix}")

# Visualize training results
train_dirs = glob("runs/detect/train*")
if train_dirs:
    latest_dir = max(train_dirs, key=lambda x: int(x.split('train')[-1]) if x.split('train')[-1].isdigit() else 0)
    results_dir = Path(latest_dir)
    print(f"Using latest training directory: {results_dir}")
    visualize_training_results(results_dir)

Using latest training directory: runs\detect\train14
Available columns in the results CSV:
['epoch', 'time', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss', 'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'val/box_loss', 'val/cls_loss', 'val/dfl_loss', 'lr/pg0', 'lr/pg1', 'lr/pg2']


<Figure size 1500x1000 with 4 Axes>

<Figure size 1200x800 with 1 Axes>

<Figure size 1000x800 with 1 Axes>

## H. Testing

In [ ]:
# Define constant for prediction directory
PREDICT_DIR = "runs/detect"

def detect_video(video_path: Path, model_path: Union[Path, str], conf_threshold: float = 0.25):
    """
    Run object detection on a video using a trained YOLO model
    
    Args:
        video_path: Path to input video
        model_path: Path to trained YOLO model
        conf_threshold: Confidence threshold for detections
    
    Returns:
        Path to output video
    """
    try:
        # Load the model
        model: YOLO = YOLO(model_path)
        print(f"Loaded model from {model_path}")
        
        # Process the video
        print(f"Processing video: {video_path}")
        results = model.predict(
            source=str(video_path),
            conf=conf_threshold,
            save=True,
            device=0 if cuda.is_available() else 'cpu',
            project=PREDICT_DIR,  # Specify project directory
            name="video_prediction"  # Specify run name
        )
        
        # Iterate through results to trigger processing
        frame_count = 0
        for result in results:
            frame_count += 1
            if frame_count % 10 == 0:
                print(f"Processed {frame_count} frames", end="\r")
        
        print(f"\nProcessed {frame_count} total frames")
        
        # Find the actual output directory
        predict_dirs = list(Path(PREDICT_DIR).glob("video_prediction*"))
        if predict_dirs:
            # Get the latest prediction directory
            latest_predict_dir = max(predict_dirs, key=lambda x: x.stat().st_mtime)
            
            # Look for video files in the output directory
            video_files = list(latest_predict_dir.glob("*.avi")) + list(latest_predict_dir.glob("*.mp4"))
            
            if video_files:
                output_video = video_files[0]
                print(f"✅ Detection completed, output saved to: {output_video}")
                return output_video
            else:
                print(f"⚠️ No output video found in {latest_predict_dir}")
                # Check for individual frame outputs
                frame_files = list(latest_predict_dir.glob("*.jpg")) + list(latest_predict_dir.glob("*.png"))
                if frame_files:
                    print(f"Found {len(frame_files)} output frames instead")
                    return latest_predict_dir
                return None
        else:
            print("❌ No prediction output directory found")
            return None
            
    except Exception as e:
        print(f"❌ Error running detection: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

def display_detection_results(output_path):
    """Display detection results - either video or frames"""
    if output_path is None:
        print("❌ No output to display")
        return
        
    if output_path.is_file() and output_path.suffix in ['.mp4', '.avi']:
        # Display video
        display_detection_video(output_path)
    elif output_path.is_dir():
        # Display sample frames
        frame_files = list(output_path.glob("*.jpg")) + list(output_path.glob("*.png"))
        if frame_files:
            print(f"Displaying {min(5, len(frame_files))} sample detection frames:")
            for i, frame_path in enumerate(sorted(frame_files)[:5]):
                plt.figure(figsize=(12, 8))
                img = plt.imread(frame_path)
                plt.imshow(img)
                plt.title(f"Detection Frame {i+1}: {frame_path.name}")
                plt.axis('off')
                plt.tight_layout()
                plt.show()
        else:
            print("❌ No frames found in output directory")

def display_detection_video(video_path: Path):
    """Display video with detections in notebook"""
    try:
        if not video_path.exists():
            print(f"❌ Video file not found: {video_path}")
            return
            
        file_size = video_path.stat().st_size / (1024 * 1024)  # Size in MB
        print(f"Video file size: {file_size:.2f} MB")
        
        # For large files, show a warning
        if file_size > 50:
            print("⚠️ Large video file, displaying may be slow")
            
        with open(video_path, "rb") as video_file:
            data = video_file.read()
            video_base64 = b64encode(data).decode()
            
            video_html = f"""
            <video width="640" height="480" controls>
                <source src="data:video/mp4;base64,{video_base64}" type="video/mp4">
                Your browser does not support the video tag.
            </video>
            """
            display(HTML(video_html))
            
    except Exception as e:
        print(f"❌ Error displaying video: {str(e)}")
        # Fallback: try to show some frames from the same directory
        parent_dir = video_path.parent
        frame_files = list(parent_dir.glob("*.jpg"))
        if frame_files:
            print("Showing sample frames instead:")
            for i, frame_path in enumerate(frame_files[:3]):
                plt.figure(figsize=(10, 6))
                img = plt.imread(frame_path)
                plt.imshow(img)
                plt.title(f"Frame {i+1}")
                plt.axis('off')
train_dirs = glob(f"{PREDICT_DIR}/train*")
if train_dirs:
    # Get latest training directory by number (same as visualization)
    latest_dir = max(train_dirs, key=lambda x: int(x.split('train')[-1]) if x.split('train')[-1].isdigit() else 0)
    trained_model_path = Path(latest_dir) / "weights" / "best.pt"
    
    if trained_model_path.exists():
        print(f"Using latest model from {latest_dir}: {trained_model_path}")
    else:
        print(f"⚠️ Model not found in {latest_dir}, looking for alternatives...")
        # Fallback to modification time method
        possible_paths = list(Path(PREDICT_DIR).glob("train*/weights/best.pt"))
        if possible_paths:
            trained_model_path = max(possible_paths, key=lambda x: x.stat().st_mtime)
            print(f"Using latest model by modification time: {trained_model_path}")
        else:
            print("❌ No trained model found, using default YOLO model")
            trained_model_path = "yolo11m.pt"
else:
    print("❌ No training directories found, using default YOLO model")
    trained_model_path = "yolo11m.pt"

# Clean up old prediction directories (optional)
old_predict_dirs = list(Path(PREDICT_DIR).glob("video_prediction*"))
if len(old_predict_dirs) > 3:  # Keep only last 3 runs
    for old_dir in sorted(old_predict_dirs, key=lambda x: x.stat().st_mtime)[:-3]:
        import shutil
        shutil.rmtree(old_dir, ignore_errors=True)
        print(f"Cleaned up old prediction: {old_dir}")
        import shutil
        shutil.rmtree(old_dir, ignore_errors=True)
        print(f"Cleaned up old prediction: {old_dir}")

# Run detection on the video
video_path = dataset_dir / "locker.mp4"
print(f"Input video: {video_path}")
print(f"Video exists: {video_path.exists()}")

if video_path.exists():
    output_result = detect_video(
        video_path=video_path, 
        model_path=trained_model_path,
        conf_threshold=3.0
    )
    
    # Display the result
    display_detection_results(output_result)
else:
    print(f"❌ Input video not found: {video_path}")

Using latest model from runs/detect\train14: runs\detect\train14\weights\best.pt
Cleaned up old prediction: runs\detect\video_prediction
Input video: dataset\locker.mp4
Video exists: True


Loaded model from runs\detect\train14\weights\best.pt
Processing video: dataset\locker.mp4

WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/949) d:\Salman\Kerja\Pantona\Locker\dataset\locker.mp4: 96x128 98 opens, 24 closes, 278.3ms
video 1/1 (frame 2/949) d:\Salman\Kerja\Pantona\Locker\dataset\locker.mp4: 96x128 100 opens, 25 closes, 127.0ms
video 1/1 (frame 3/949) d:\Salman\Kerja\Pantona\Locker\dataset\locker.mp4: 96x128 103 opens, 24 closes, 134.7ms
video 1/1 (frame 4/949) d:\Salman